<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Итоговый-вывод" data-toc-modified-id="Итоговый-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Итоговый вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Цель исследования** - выполнить следующие задачи:

1. Загрузить и изучить данные.
2. Умножение матриц. Ответить на вопрос и обосновать решение.
3. Предложить алгоритм преобразования данных.
4. Запрограммировать алгоритм.

**Ход исследования**

Данные находятся в файле `/datasets/insurance.csv`. О качестве данных нам ничего не известно. Значит перед исследовательской работой понадбится просмотреть данные. Проверим общую информацию, если понадобится, то проведём предобработку данных, чтобы исправить ошибки.

После изучения данных займёмся умножением матриц и дадим ответ на главный вопрос (Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?). Создадим алгоритм для преобразования и проверим его.

## Загрузка данных

In [1]:
#подключим необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
#прочитаем файл
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
#выведем первые 5 строк таблицы
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
#выведем общую информацию
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
#оценим данные в таблице методом describe
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
#проверим колличество пропусков в %
pd.DataFrame(round(data.isna().mean()*100,)).style.background_gradient('coolwarm')

,0
Пол,0.000000
Возраст,0.000000
Зарплата,0.000000
Члены семьи,0.000000
Страховые выплаты,0.000000


In [7]:
#кол-во уникальных значений в таблице
data.nunique()

Пол                    2
Возраст               46
Зарплата             524
Члены семьи            7
Страховые выплаты      6
dtype: int64

In [8]:
#проверим наличие дубликатов
data.duplicated().sum()

153

In [9]:
#удалим дубликаты
data = data.drop_duplicates()

In [10]:
#изменим тип столбцов Возраст и Зарплата на int
data['Возраст'] = data['Возраст'].astype(int)
data['Зарплата'] = data['Зарплата'].astype(int)
data.dtypes

Пол                  int64
Возраст              int64
Зарплата             int64
Члены семьи          int64
Страховые выплаты    int64
dtype: object

**Вывод**

При изучении данных мы выяснили:

1. Каждый объект таблицы - данные о застрахованном лице. Его пол, возраст, зарплата, количество членов в семье и сколько раз получал страховую выплату. 
2. Пропуски в данных отсутствуют.
3. Присутствует небольшое количество дубликатов. Мы их удалили.
4. В столбце Возраст и Зарплата для удобства можно изменить тип данных на int.
5. Некоторые люди получали выплату по страховке до 5 раз.

Можно приступить к умножению матриц.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос**

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
* а) Изменится. Приведите примеры матриц.
* б) Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной

**Ответ:** не изменится

**Обоснование:** 

1. Возьмём новую матрицу $X1$

$X1 = X * P$ ,где $X$ - матрица признаков, $P$ - обратимая матрица

2. Подставим $X1$ в формулу $w$ и назовём её $w1$

$w1 = ((XP)^T XP)^{-1} (XP)^Ty$

3. Раскроем $(XP)^T$ используя свойство транспонированной матрицы $(AB)^T = B^TA^T$

$w1 = (P^TX^T XP)^{-1} P^TX^Ty$

4. Перегруппируем множители в скобках $(P^TX^T XP)^{-1}$ 

$w1 = (P^T(X^T X)P)^{-1} P^TX^Ty$

5. Раскроем скобки $(P^T(X^T X)P)^{-1}$ используя свойство обратной матрицы $(AB)^{-1} = B^{-1}A^{-1}$

$w1 = P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^Ty$

6. Поскольку $P$ - обратимая матрица, то можно использовать следующее свойство $PP^{-1} = (P^T)^{-1}P^T = E$ ,где $E$ - единичная матрица.

$w1 = P^{-1}(X^TX)^{-1}EX^Ty = P^{-1}(X^TX^{-1})X^Ty$

7. Можно заметить, что справа от $(P)^{-1}$ мы получили $w$

$w1 = P^{-1}(X^TX)^{-1}X^Ty = P^{-1}w$

8. Теперь подставим $X1$ и $w1$ в формулу $a$ и назовём её $a1$

$a1 = X1w1 = XPP^{-1}w$

9. Поскольку $P$ - обратимая матрица, то $PP^{-1} = E$. Получаем следующее

$a1 = XEw = Xw = a$

10. Из результата видно, что умножение матрицы признаков на обратимую матрицу ничего не изменит.

**Вывод**

Умножение признаков на обратимую матрицу не изменит качество линейно регрессии. Можно приступить к алгоритму преобразования 

## Алгоритм преобразования

**Алгоритм**

Для шифрования данных будем умножать матрицу признаков $X$ на обратимую матрицу $P$. 

Алгоритм будет следующим:

1. Создадим матрицу $P$ со случайными числами по нормальному распределению.
2. Проверим, что она получилась обратимой. Если нет, повторим генерацию, пока не получим обратимую матрицу. Но вероятность получить необратимую крайне мала.
3. Умножим матрицу признаков $X$ на матрицу $P$.

**Обоснование**

Выше мы доказали, что предсказания модели не поменяются при преобразовании. Значит можно сделать вывод, что качество модели тоже не должно измениться. Также мы знаем, что качество модели - это производная от предсказаний, а они у нас одинаковые до и после преобразования, и целевого признака.

Проверим наше предположение ниже.

## Проверка алгоритма

In [11]:
#создадим случайную квадратную матрицу 4 на 4 (равная количеству признаков)
random_matrix = np.random.randint(100, size=(4,4))
random_matrix

array([[69, 15,  9, 34],
       [60, 28, 59, 58],
       [76, 13, 27, 38],
       [19, 20, 16, 79]])

In [12]:
#проверим на обратимость
inv_matrix = np.linalg.inv(random_matrix)
inv_matrix

array([[ 0.005156  , -0.0061569 ,  0.01450667, -0.00467667],
       [ 0.12024464,  0.05469682, -0.14705841, -0.02117106],
       [-0.03891104,  0.01174866,  0.02730517, -0.00501321],
       [-0.02380102, -0.014746  ,  0.02821087,  0.02015809]])

Проверку на обратимость матрица прошла

In [13]:
#подготовим признаки
features = data.drop('Страховые выплаты', axis = 1)
target = data['Страховые выплаты']

In [14]:
#масштабирование
scaler = StandardScaler()
scaler.fit(features)
features = scaler.transform(features)

In [15]:
#посмотрим на шифрованную матрицу
cipher_matrix = features @ random_matrix
cipher_matrix

array([[ 210.18118663,   56.90615791,  101.69090173,  124.63154938],
       [  19.11998881,   28.27914049,   87.04118603,   46.59653989],
       [-247.92141475,  -68.17387993,  -91.73177608, -206.27433644],
       ...,
       [-178.64016788,  -44.6370973 ,  -90.23981204,  -74.79539174],
       [ -18.35059357,    8.60269712,  -47.02077955,   74.21281128],
       [  49.68463192,    2.28469601,  -13.04975465,    1.49475198]])

Можно заметить, что в данных теперь хранится непонятное число.

Теперь можно подкрепить теоретическое обоснование практикой, чтобы точно быть уверенными, что мы делаем всё правильно. Рассчитаем $w$ и $a$ по исходным признакам, и $w1$ и $a1$ по преобразованным признакам.

In [16]:
w = np.linalg.inv(features.T.dot(features)).dot(features.T).dot(target)
a = features.dot(w)
w1 = np.linalg.inv(cipher_matrix.T.dot(cipher_matrix)).dot(cipher_matrix.T).dot(target)
a1 = cipher_matrix.dot(w1)

display(a, a1)
print((a-a1).sum())

array([ 0.36706966,  0.54004184, -0.05813168, ..., -0.41390424,
       -0.34620098, -0.1018115 ])

array([ 0.36706966,  0.54004184, -0.05813168, ..., -0.41390424,
       -0.34620098, -0.1018115 ])

-1.7982089341866958e-14


Исходя из результата можно смело сказать, что $a$ почти равен $a1$, ибо сумма расхождений очень маленькая. Значит результат предсказания не изменится.

Приступим к созданию модели.

In [17]:
#создадим модель линейной регрессии и проверим её R2 на исходных данных
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print('R2 =', r2_score(target, predictions))

R2 = 0.4302010046633359


In [18]:
#создадим модель линейной регрессии и проверим её R2 на преобразованных данных
model = LinearRegression()
model.fit(cipher_matrix, target)
predictions = model.predict(cipher_matrix)
print('R2 =', r2_score(target, predictions))

R2 = 0.4302010046633359


Можно смело сказать, что наш простейший алгоритм преобразования сработал хорошо. Качество регрессии не изменилось. 

## Итоговый вывод

В ходе работы мы:

1. Загрузили и изучили данные страховой компании «Хоть потоп».
2. Ответили и обосновали вопрос (Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?). Ответ - не изменится
3. Предложили простейший алгоритм преобразования и обосновали его.
4. Проверили наш алгоритм на модели.

**Итог**

Нам предоставили 1 файл с данными. Данные были в хорошем состоянии. Им практически не понадобилась предобработка. После изучения данных мы ответили на вопрос по умножению матриц и предложили алгоритм преобразования. После проверки алгоритма на исходных и преобразованных данных можно смело сказать, что алгоритм справился хорошо. Мы защитили данные пользователей и не потеряли при этом качество модели.